<a href="https://colab.research.google.com/github/ovapichi/laboratorio-automatico/blob/main/An%C3%A1lisis_de_Celda_de_Carga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Requiere de 4 valores de tipo float en el siguiente orden:
1.Cuadriceps Derechos
2.Cuadriceps Izquierdos
3.Isquios Derechos
4.Isquios Izquierdos

In [1]:
#!git clone "https://github.com/ovapichi/laboratorio-automatico.git"

Cloning into 'laboratorio-automatico'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 55 (delta 26), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (55/55), 18.90 KiB | 6.30 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [ ]:
#%run "/content/laboratorio-automatico/Fpdf.ipynb"

In [ ]:
import fpdf as FPDF
from datetime import datetime
import pandas as pd
import numpy as np
from fpdf import FPDF
from google.colab import drive, files
import shutil
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
class Celda:
  def __init__(self,cuadriceps_der,cuadriceps_izq,isquios_der,isquios_izq):
    #Constructor
    self.cuadriceps_der=cuadriceps_der
    self.cuadriceps_izq=cuadriceps_izq
    self.isquios_der=isquios_der
    self.isquios_izq=isquios_izq

  def calcular_ratios(self, valor1, valor2):
    #calcula los ratios de cada grupo muscular
    return round(valor1/valor2,2)

  def procesar_simetria(self):
    #Hace una recomendación en función del ratio que obtiene de la función anterior.
    referencia_data=[['Menor a 0.9', 'Debilidad Derecha'],['Entre 0.9 y 1.1', 'Deseable'], ['Mayor a 1.1', 'Debilidad Izquierda']]
    referencia_head=['Referencia', '']
    abs_data=['Extensores Derechos', self.cuadriceps_der], ['Extensores Izquierdos', self.cuadriceps_izq], ['Flexores Derechos', self.isquios_der], ['Flexores Izquierdos', self.isquios_izq]
    abs_head=['Grupo Muscular', 'Valor en kilogramos']
    ratio_cuadriceps=self.calcular_ratios(self.cuadriceps_der,self.cuadriceps_izq)
    ratio_isquios=self.calcular_ratios(self.isquios_der,self.isquios_izq)
    pdf.add_page()
    pdf.set_font("Arial",size=10)
    caratula(jugadora,"Evaluación de la fuerza isométrica de MMII con celda de cargas")
    pdf.add_page()
    pdf.set_y(4)
    pdf.cell(0,1,f'Tabla de valores absolutos',ln=True, align='l')
    pdf.add_table(abs_head,abs_data)
    pdf.set_font('Arial', 'B', 14)
    pdf.cell(0,1,f'Valores de simetría interpierna',ln=True,align='l')
    pdf.set_font('Arial', '', 12)
    pdf.cell(0,1,f'Su valor de simetría de extensores es {ratio_cuadriceps}.',ln=True, align='l')
    pdf.cell(0,1,f'Su valor de simetría de flexores es {ratio_isquios}.',ln=True, align='l')
    pdf.ln()
    pdf.add_table(referencia_head,referencia_data)
    pdf.ln()

    #print(tabulate(absolutos, headers='firstrow', tablefmt='fancy_grid'))
    #print(f'Su valor de simetría de extensores es {ratio_cuadriceps}. \nSu valor de simetría de flexores es {ratio_isquios}.')
    #print(tabulate(referencia_simetria, headers='firstrow', tablefmt="fancy_grid"))
    return ratio_cuadriceps, ratio_isquios

  def procesar_ratios(self):
    #Toma los ratios anteroposteriores y devuelve la recomendación en función de ellos. Si queda positivo el cuadriceps es mayor.
    ratio_derecho=self.calcular_ratios(self.isquios_der,self.cuadriceps_der)
    ratio_izquierdo=self.calcular_ratios(self.isquios_izq,self.cuadriceps_izq)


    ratio_data=[['Mayor a 0.6', 'Deseable'], ['Menor a 0.6', 'Debilidad de flexores']]
    ratio_head=['Referencia', '']

    pdf.add_page()
    pdf.set_y(4)
    pdf.set_font('Arial', 'B', 14)
    pdf.cell(0,1,f'Valores de ratio intrapierna.',ln=True,align='l')
    pdf.set_font('Arial', '', 12)
    pdf.cell(0,1,f'Su H:Q ratio derecho es {ratio_derecho}.',ln=True, align='l')
    pdf.cell(0,1,f'Su H:Q ratio izquierdo es {ratio_izquierdo}.',ln=True, align='l')
    pdf.ln()
    pdf.add_table(ratio_head,ratio_data)
    pdf.ln()

    return ratio_derecho, ratio_izquierdo

  def ejecutar(self):

    lista=self.procesar_simetria(), self.procesar_ratios()
    lista_nueva=[self.cuadriceps_der, self.cuadriceps_izq, self.isquios_der, self.isquios_izq]
    for i in lista:
      for e in i:
        lista_nueva.append(e)
    print(lista_nueva)

In [ ]:
# pdf = PDF("p","cm","A4")
# jugadora="anita"
# anita=Celda(50, 55, 30, 35)
# anita.ejecutar()

[50, 55, 30, 35, 0.91, 0.86, 0.6, 0.64]


In [ ]:
#exportar("hola","/content/sample_data")